In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict
import re
import os

In [7]:
script_links = []
with urlopen('https://www.livesinabox.com/friends/scripts.shtml') as res:
    soup = BeautifulSoup(res, 'html.parser')
    for anchor in soup.find_all('a'):
        if 'Episode ' in anchor.get_text() and any(s not in anchor.get_text() for s in ['Guide']):
            # print(anchor.get('href'))
            script_links.append(anchor.get('href'))

In [8]:
BASE = 'https://www.livesinabox.com/friends/'

def get_season(link) -> int:
    if 'season' in link:
        season_num = int(link.lstrip("season").split("/")[0])
    else:
        season_num = 10
    return season_num


def get_episode(link) -> int:
    if link == 'season9/0923-0924.html':
        episode = 23
    if 'season' in link:
        episode = int(re.search('/\d*', link).group()[2:])
    else:
        episode = int(link[2:4])
    return episode

season_labeled_links = defaultdict(list)
for link in script_links:
    season_labeled_links[get_season(link)].append((get_episode(link), link))
print(season_labeled_links.values())

dict_values([[(1, 'season1/101pilot.htm'), (2, 'season1/102towsg.htm'), (3, 'season1/103thumb.htm'), (4, 'season1/104towgs.htm'), (5, 'season1/105egld.htm'), (6, 'season1/106butt.htm'), (7, 'season1/107towbo.htm'), (8, 'season1/108ndt.htm'), (9, 'season1/109uga.htm'), (10, 'season1/110monk.htm'), (11, 'season1/111mbing.htm'), (12, 'season1/112towdl.htm'), (13, 'season1/113tits.htm'), (14, 'season1/114towch.htm'), (15, 'season1/115towsg.htm'), (16, 'season1/116part1.htm'), (17, 'season1/117part2.htm'), (18, 'season1/118poke.htm'), (19, 'season1/119mga.htm'), (20, 'season1/120toweo.htm'), (21, 'season1/121towfm.htm'), (22, 'season1/122ick.htm'), (23, 'season1/123birth.htm'), (24, 'season1/124rafo.htm')], [(1, 'season2/201rng.htm'), (2, 'season2/202brst.htm'), (3, 'season2/203towhd.htm'), (4, 'season2/204towph.htm'), (5, 'season2/205fsae.htm'), (6, 'season2/206baby.htm'), (7, 'season2/207rofo.htm'), (8, 'season2/208list.htm'), (9, 'season2/209towpd.htm'), (10, 'season2/210russ.htm'), (11,

In [9]:
DEST_DIR = './data/'

for season_num in season_labeled_links.keys():
    try:
        os.mkdir(DEST_DIR + "season" + str(season_num))
    except Exception as e:
        print(e)

In [10]:
for season_num in season_labeled_links.keys():
    for _, (episode_num, script_link) in enumerate(season_labeled_links[season_num]):
        with urlopen(BASE + script_link) as res:
            soup = BeautifulSoup(res, 'html.parser')
            lines = soup.find('body').get_text().split('\n')
            lines = [line for line in lines if not any(hay in line for hay in [
                'Written by:',
                'Transcribed by:',
                'Additional transcribing by:',
                '(Note: The previously unseen parts of this episode are shown in blue text.)'
            ])]
            text = '\n'.join(lines)
            print(f"S{season_num} E{episode_num}: EXTRACTED")
            with open(DEST_DIR + f"season{str(season_num)}/episode{str(episode_num)}.txt", 'w') as f:
                f.write(text)

S1 E1: EXTRACTED
S1 E2: EXTRACTED
S1 E3: EXTRACTED
S1 E4: EXTRACTED
S1 E5: EXTRACTED
S1 E6: EXTRACTED
S1 E7: EXTRACTED
S1 E8: EXTRACTED
S1 E9: EXTRACTED
S1 E10: EXTRACTED
S1 E11: EXTRACTED
S1 E12: EXTRACTED
S1 E13: EXTRACTED
S1 E14: EXTRACTED
S1 E15: EXTRACTED
S1 E16: EXTRACTED
S1 E17: EXTRACTED
S1 E18: EXTRACTED
S1 E19: EXTRACTED
S1 E20: EXTRACTED
S1 E21: EXTRACTED
S1 E22: EXTRACTED
S1 E23: EXTRACTED
S1 E24: EXTRACTED
S2 E1: EXTRACTED
S2 E2: EXTRACTED
S2 E3: EXTRACTED
S2 E4: EXTRACTED
S2 E5: EXTRACTED
S2 E6: EXTRACTED
S2 E7: EXTRACTED
S2 E8: EXTRACTED
S2 E9: EXTRACTED
S2 E10: EXTRACTED
S2 E11: EXTRACTED
S2 E12: EXTRACTED
S2 E14: EXTRACTED
S2 E15: EXTRACTED
S2 E16: EXTRACTED
S2 E17: EXTRACTED
S2 E18: EXTRACTED
S2 E19: EXTRACTED
S2 E20: EXTRACTED
S2 E21: EXTRACTED
S2 E22: EXTRACTED
S2 E23: EXTRACTED
S2 E24: EXTRACTED
S3 E1: EXTRACTED
S3 E2: EXTRACTED
S3 E3: EXTRACTED
S3 E4: EXTRACTED
S3 E5: EXTRACTED
S3 E6: EXTRACTED
S3 E7: EXTRACTED
S3 E8: EXTRACTED
S3 E9: EXTRACTED
S3 E10: EXTRACTED
S